# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [31]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer

from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

import pickle
import shap

c:\Users\hmwan\dsi\production\production-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


# Get X and Y

Create the features data frame and target data.

In [4]:
X = fires_dt.drop(columns=["area"])
y = fires_dt["area"]

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (517, 12)
y shape: (517,)


In [5]:
X.head(), y.head()

(   coord_x  coord_y month  day  ffmc   dmc     dc  isi  temp  rh  wind  rain
 0        7        5   mar  fri  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0
 1        7        4   oct  tue  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0
 2        7        4   oct  sat  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0
 3        8        6   mar  fri  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2
 4        8        6   mar  sun  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0,
 0    0.0
 1    0.0
 2    0.0
 3    0.0
 4    0.0
 Name: area, dtype: float64)

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [34]:
# Identify feature types
num_cols = X.select_dtypes(include=["int64", "float64"]).columns
cat_cols = X.select_dtypes(include=["object"]).columns

print("Numeric columns:", list(num_cols))
print("Categorical columns:", list(cat_cols))

Numeric columns: ['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain']
Categorical columns: ['month', 'day']


In [35]:
numeric_pipe_1 = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())   # you can switch to RobustScaler() or MinMaxScaler()
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preproc1 = ColumnTransformer([
    ("num", numeric_pipe_1, num_cols),
    ("cat", categorical_pipe, cat_cols)
])

### Preproc 2

Create preproc1 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [11]:
numeric_pipe_2 = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("transform", FunctionTransformer(np.log1p, feature_names_out="one-to-one")),
    ("scaler", StandardScaler())
])

preproc2 = ColumnTransformer([
    ("num", numeric_pipe_2, num_cols),
    ("cat", categorical_pipe, cat_cols)
])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [12]:
# Pipeline A = preproc1 + baseline
pipe_A = Pipeline([
    ("preprocessing", preproc1),
    ("regressor", Ridge(random_state=42))
])

In [13]:
# Pipeline B = preproc2 + baseline
pipe_B = Pipeline([
    ("preprocessing", preproc2),
    ("regressor", Ridge(random_state=42))
])

In [14]:
# Pipeline C = preproc1 + advanced model
pipe_C = Pipeline([
    ("preprocessing", preproc1),
    ("regressor", RandomForestRegressor(random_state=42))
])

In [15]:
# Pipeline D = preproc2 + advanced model
pipe_D = Pipeline([
    ("preprocessing", preproc2),
    ("regressor", RandomForestRegressor(random_state=42))
])  

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [23]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [20]:
# Pipeline A
param_grid_A = {
    "regressor__alpha": [0.01, 0.1, 1, 10, 100, 300]
}

# Pipeline B
param_grid_B = {
    "regressor__alpha": [0.01, 0.1, 1, 10, 100, 300]
}

# Pipeline C
param_grid_C = {
    "regressor__n_estimators": [200, 500],
    "regressor__max_depth": [None, 10],
    "regressor__min_samples_split": [2, 5]
}

# Pipeline D
param_grid_D = {
    "regressor__n_estimators": [200, 500],
    "regressor__max_depth": [None, 10],
    "regressor__min_samples_split": [2, 5]
}

In [24]:
pipelines = {
    "A_preproc1_baseline": (pipe_A, param_grid_A),
    "B_preproc2_baseline": (pipe_B, param_grid_B),
    "C_preproc1_advanced": (pipe_C, param_grid_C),
    "D_preproc2_advanced": (pipe_D, param_grid_D),
}

best_models = {}
rows = []

for name, (pipe, grid) in pipelines.items():
    gs = GridSearchCV(
        estimator=pipe,
        param_grid=grid,
        scoring=rmse_scorer,
        cv=5,
        n_jobs=-1
    )
    gs.fit(X_train, y_train)

    best_models[name] = gs.best_estimator_

    rows.append({
        "Pipeline": name,
        "Tried combos": len(gs.cv_results_["params"]),
        "Best params": gs.best_params_,
        "Best CV RMSE": -gs.best_score_  # flip sign back
    })

results_df = pd.DataFrame(rows).sort_values("Best CV RMSE")
results_df

,Pipeline,Tried combos,Best params,Best CV RMSE
1,B_preproc2_baseline,6,{'regressor__alpha': 300},38.716132
0,A_preproc1_baseline,6,{'regressor__alpha': 300},38.779066
2,C_preproc1_advanced,8,"{'regressor__max_depth': None, 'regressor__min...",47.674215
3,D_preproc2_advanced,8,"{'regressor__max_depth': None, 'regressor__min...",47.708526


In [25]:
best_pipeline_name = results_df.iloc[0]["Pipeline"]
best_model = best_models[best_pipeline_name]

y_pred = best_model.predict(X_test)

test_rmse = rmse(y_test, y_pred)
test_mae = mean_absolute_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print("Best pipeline (by CV RMSE):", best_pipeline_name)
print("Test RMSE:", test_rmse)
print("Test MAE :", test_mae)
print("Test R^2 :", test_r2)

Best pipeline (by CV RMSE): B_preproc2_baseline
Test RMSE: 108.49015260770499
Test MAE : 24.235504967763895
Test R^2 : 0.0014972229407249626


# Evaluate

+ Which model has the best performance?

Pipeline B (preprocess B + Ridge) is the best

# Export

+ Save the best performing model to a pickle file.

In [26]:
with open("best_forestfire_model.pkl", "wb") as f:
    pickle.dump(best_model, f)

print("Saved: best_forestfire_model.pkl")

Saved: best_forestfire_model.pkl


# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

In [33]:
preproc = best_model.named_steps["preprocessing"]
reg = best_model.named_steps["regressor"]

X_train_t = preproc.transform(X_train)
X_test_t = preproc.transform(X_test)

feature_names = preproc.get_feature_names_out()

is_tree = hasattr(reg, "estimators_") or reg.__class__.__name__.lower().find("forest") >= 0
is_linear = hasattr(reg, "coef_")

if is_tree:
    explainer = shap.TreeExplainer(reg)
    shap_values = explainer.shap_values(X_train_t)
    base_value = explainer.expected_value
elif is_linear:
    # Works well for linear models; use a small background for speed
    background = X_train_t[:200]
    explainer = shap.LinearExplainer(reg, background)
    shap_values = explainer.shap_values(X_train_t)
    base_value = explainer.expected_value
else:
    # Fallback (slow). Use a small background/sample.
    background = X_train_t[:100]
    f = lambda z: reg.predict(z)
    explainer = shap.KernelExplainer(f, background)
    shap_values = explainer.shap_values(X_train_t[:300])
    base_value = explainer.expected_value

i = 0  # change this index if you want a different test row
x_i = X_test_t[i]
pred_i = reg.predict(X_test_t[[i]])[0]

print("Selected test index:", i)
print("Prediction for this observation:", pred_i)

# SHAP values for this observation (need matching array depending on explainer)
if is_tree or is_linear:
    sv_i = shap.TreeExplainer(reg).shap_values(X_test_t[[i]]) if is_tree else explainer.shap_values(X_test_t[[i]])
    sv_i = sv_i[0]
else:
    sv_i = explainer.shap_values(X_test_t[[i]])[0]

local = pd.DataFrame({
    "feature": feature_names,
    "shap_value": sv_i
}).assign(abs_shap=lambda d: d["shap_value"].abs()).sort_values("abs_shap", ascending=False)

local.head(10)

Selected test index: 0
Prediction for this observation: 6.91224769214057


,feature,shap_value,abs_shap
7,num__rh,-2.814740,2.814740
3,num__dmc,-2.546223,2.546223
0,num__coord_x,1.682867,1.682867
6,num__temp,-1.559106,1.559106
1,num__coord_y,0.500455,0.500455
4,num__dc,-0.480316,0.480316
8,num__wind,0.400003,0.400003
24,cat__day_sat,0.396508,0.396508
11,cat__month_aug,-0.166961,0.166961
22,cat__day_fri,0.130428,0.130428


*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.